# Statistics of MFCC on Laughter data

* [Summary](#Summary)

----
## Summary

* 

In [1]:
# Load data and as well as necessary modules
import csv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from scipy import stats
import pandas as pd
from IPython.display import display

with open("NewData.csv", 'rt') as f:
    #f = open("MartixOfWindowAverageAllMFCC.csv", 'rt')
    data = []
    laugh_data = []
    talk_data = []
    try:
        reader = csv.reader(f)
        #header = next(reader)
        for row in reader:
            data.append(np.array(row))
            if(row[-1] == '1'):
                laugh_data.append(row)
            else:
                talk_data.append(row)                    
            #data.append(row)
    finally:
        f.close()
    
#header = np.array(header)
header = np.array(['MFC0','MFC1','MFC2','MFC3','MFC4','MFC5','MFC6','MFC7','MFC8',\
                   'MFC9','MFC10','MFC11','MFC12', 
                  ])
data = np.array(data) # data[patient, datafield] # all data is strings, even numbers
laugh_data = np.array(laugh_data,float)
talk_data = np.array(talk_data,float)
features = np.array(range(13))
Xorig_nd = np.array(data[:,features],float)
n,d = Xorig_nd.shape
print(Xorig_nd.shape)
y_n = np.array(data[:,-1],float)

#df_features = pd.DataFrame(columns=header[key_stats])
#display(df_features)
display(pd.DataFrame(header[features],columns=['features']))


(40021, 13)


,features
0,MFC0
1,MFC1
2,MFC2
3,MFC3
4,MFC4
5,MFC5
6,MFC6
7,MFC7
8,MFC8
9,MFC9


### KNN

using the MFCC's only try logistic regression classification
  -try using all MFCC features
  - first no regularization, 
  - with L1 regularization

-have a for loop for trying cross validation
-have a loop for checking regularization



## Prediction
Logistic Regression (maximum entropy)

In [2]:
from __future__ import print_function

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print(__doc__)

# Loading the Digits dataset
#digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
#n_samples = len(digits.images)
#X = digits.images.reshape((n_samples, -1))
#y = digits.target

#------------------------------------------
X = Xorig_nd[:,features]
y = y_n

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=0)

# MINH ADJUST TO USE MORE DATA TO RUN OVERNIGHT

# Set the parameters by cross-validation
#tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                     'C': [1, 10, 100, 1000]},
#                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.805 (+/-0.009) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.788 (+/-0.005) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.816 (+/-0.012) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.805 (+/-0.009) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.842 (+/-0.013) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.814 (+/-0.012) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.880 (+/-0.013) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.820 (+/-0.011) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

        0.0       0.84      0.92      0.88      2955
    

In [14]:
from operator import itemgetter
from itertools import groupby

#-----------Audio1----------------
with open("S_Deception_audio1.csv", 'rt') as f:
    data = []
    try:
        reader = csv.reader(f)
        for row in reader:
            data.append(np.array(row))                   
    finally:
        f.close()
    
data = np.array(data,float) 
prediction=clf.predict(data)

laugh_moments=[]
for i in range (0,len(data)):
    if(prediction[i]==1):
        laugh_moments.append(i)
        
#laugh_moments=np.array(laugh_moments)*0.05
#print(laugh_moments)

def myfunc(lst):
    ret = []
    a = b = lst[0]                           # a and b are range's bounds

    for el in lst[1:]:
        if el == b+1: b = el                 # range grows
        else:                                # range ended
            ret.append(a if a==b else (a,b)) # is a single or a range?
            a = b = el                       # let's start again with a single
    ret.append(a if a==b else (a,b))         # corner case for last single/range
    return ret

ret=myfunc(laugh_moments)

for i in range (0,len(ret)):
    if(isinstance( ret[i], int )):
        current=ret[i]
        start_time=current*0.05
        end_time=start_time+0.3
        print(round(start_time,2),round(end_time,2))
    else:
        temp=list(ret[i])
        start_time=round(temp[0]*0.05,2)
        end_time=round(temp[1]*0.05+0.3,2)
        print(start_time,end_time)

0.7 1.3
2.35 2.8
3.15 3.95
4.4 5.25
5.6 6.2
6.4 7.4
7.65 8.1
8.0 8.55
12.9 13.85
13.7 14.0
13.85 14.45
15.35 15.65
23.2 23.75
23.6 24.1
24.1 24.45
30.7 31.0
36.35 36.65
54.15 54.5
63.0 63.45
63.5 63.8
65.25 65.75
72.4 72.8
72.8 73.1
78.25 78.8
80.15 80.6
95.55 96.0
110.35 110.65
114.85 115.2
139.35 139.85
160.75 161.2
163.5 163.85
163.95 164.35
198.9 199.2
206.6 207.1
222.35 222.65
260.45 260.75
305.25 305.55
323.95 324.25
324.05 324.45
337.6 337.9
337.7 338.3


In [15]:
#-----------Audio2----------------
with open("S_Deception_audio2.csv", 'rt') as f:
    data = []
    try:
        reader = csv.reader(f)
        for row in reader:
            data.append(np.array(row))                   
    finally:
        f.close()
    
data = np.array(data,float) 
prediction=clf.predict(data)

laugh_moments=[]

for i in range (0,len(data)):
    if(prediction[i]==1):
        laugh_moments.append(i)
        
#laugh_moments=np.array(laugh_moments)*0.05
#print(laugh_moments)

def myfunc(lst):
    ret = []
    a = b = lst[0]                           # a and b are range's bounds

    for el in lst[1:]:
        if el == b+1: b = el                 # range grows
        else:                                # range ended
            ret.append(a if a==b else (a,b)) # is a single or a range?
            a = b = el                       # let's start again with a single
    ret.append(a if a==b else (a,b))         # corner case for last single/range
    return ret

ret=myfunc(laugh_moments)

for i in range (0,len(ret)):
    if(isinstance( ret[i], int )):
        current=ret[i]
        start_time=current*0.05
        end_time=start_time+0.3
        print(round(start_time,2),round(end_time,2))
    else:
        temp=list(ret[i])
        start_time=round(temp[0]*0.05,2)
        end_time=round(temp[1]*0.05+0.3,2)
        print(start_time,end_time)

0.0 4.35
4.15 4.7
4.5 4.8
4.6 5.55
5.75 7.45
7.45 9.45
9.3 11.25
11.2 11.75
11.75 12.25
12.4 13.8
13.8 14.35
14.2 14.55
14.4 15.1
15.1 15.75
15.75 16.25
16.5 16.95
16.75 17.05
16.95 18.15
18.2 18.8
18.6 18.95
19.15 20.1
20.0 20.3
20.1 20.65
20.55 21.65
22.0 23.45
23.3 23.6
23.45 25.0
24.95 28.3
28.1 28.7
29.55 30.15
29.95 30.25
30.1 30.7
31.05 31.65
31.75 32.5
32.6 33.65
33.45 34.25
34.1 34.45
34.85 35.45
35.85 36.9
36.85 38.8
39.15 40.3
40.2 41.05
40.85 41.9
41.7 42.7
42.65 43.4
43.35 43.65
43.45 44.1
43.9 44.2
44.25 45.0
44.8 45.4
45.3 45.9
46.1 46.45
46.25 46.55
46.6 47.4
47.7 48.35
48.3 49.0
49.0 49.65
50.05 50.55
50.7 51.35
51.35 51.9
51.7 53.05
52.85 53.25
53.05 53.4
53.2 53.7
53.6 54.15
54.0 54.3
54.1 54.5
54.45 55.0
55.1 55.4
55.5 55.85
56.65 58.5
58.45 59.0
59.05 59.6
59.55 60.0
60.25 60.85
61.2 61.7
62.25 62.95
63.95 64.85
65.05 66.65
67.8 68.3
68.2 70.05
70.15 70.5
70.55 70.85
70.7 71.0
71.6 71.9
72.8 73.2
73.05 73.35
74.15 74.9
74.7 75.0
75.45 76.05
75.9 76.7
77.25 77.7
78.

In [16]:
#-----------Audio3----------------
with open("S_Deception_audio3.csv", 'rt') as f:
    data = []
    try:
        reader = csv.reader(f)
        for row in reader:
            data.append(np.array(row))                   
    finally:
        f.close()
    
data = np.array(data,float) 
prediction=clf.predict(data)

laugh_moments=[]
for i in range (0,len(data)):
    if(prediction[i]==1):
        laugh_moments.append(i)
        
#laugh_moments=np.array(laugh_moments)*0.05
#print(laugh_moments)

def myfunc(lst):
    ret = []
    a = b = lst[0]                           # a and b are range's bounds

    for el in lst[1:]:
        if el == b+1: b = el                 # range grows
        else:                                # range ended
            ret.append(a if a==b else (a,b)) # is a single or a range?
            a = b = el                       # let's start again with a single
    ret.append(a if a==b else (a,b))         # corner case for last single/range
    return ret

ret=myfunc(laugh_moments)

for i in range (0,len(ret)):
    if(isinstance( ret[i], int )):
        current=ret[i]
        start_time=current*0.05
        end_time=start_time+0.3
        print(round(start_time,2),round(end_time,2))
    else:
        temp=list(ret[i])
        start_time=round(temp[0]*0.05,2)
        end_time=round(temp[1]*0.05+0.3,2)
        print(start_time,end_time)

0.0 0.6
0.55 0.85
0.7 1.0
0.95 2.85
2.7 3.45
3.6 4.1
4.2 4.8
5.1 5.4
5.45 5.85
6.1 8.95
8.75 9.15
9.2 9.5
9.3 9.6
10.55 10.85
12.3 12.65
13.25 13.55
13.35 13.75
14.15 14.55
14.4 14.7
14.95 15.25
15.5 16.05
15.95 16.25
16.3 16.8
16.65 17.15
18.7 19.2
19.4 19.7
19.55 19.95
20.45 20.95
20.9 21.25
21.35 21.8
22.15 22.75
25.55 26.05
29.1 29.4
30.0 30.3
30.15 30.6
32.0 32.3
32.4 32.85
34.4 34.85
35.5 35.8
35.7 36.05
35.85 36.15
36.95 37.25
37.35 37.65
37.55 40.0
39.8 44.45
44.45 45.5
45.45 47.9
47.7 48.0
47.85 48.45
49.2 49.9
49.8 50.1
49.9 51.05
51.25 51.6
51.4 51.9
51.9 52.35
52.4 52.8
52.9 55.35
55.3 56.0
55.9 56.4
56.3 56.6
56.4 56.95
56.8 57.1
57.05 57.35
57.15 57.75
57.8 58.25
58.5 58.8
58.7 59.05
58.85 59.4
59.45 59.8
59.85 61.25
61.4 62.95
63.2 63.5
64.4 64.75
64.65 65.4
65.25 65.55
66.0 66.6
66.5 67.4
67.2 67.55
67.8 68.25
68.65 69.35
70.7 71.0
70.85 71.45
71.25 71.95
71.8 73.5
73.75 74.05
74.05 75.75
77.2 77.5
77.9 78.25
78.55 78.85
78.95 79.35
80.55 81.15
81.05 82.15
82.0 82.4
82.

In [17]:
#-----------Audio4----------------
with open("S_Deception_audio4.csv", 'rt') as f:
    data = []
    try:
        reader = csv.reader(f)
        for row in reader:
            data.append(np.array(row))                   
    finally:
        f.close()
    
data = np.array(data,float) 
prediction=clf.predict(data)

laugh_moments=[]
for i in range (0,len(data)):
    if(prediction[i]==1):
        laugh_moments.append(i)
        
#laugh_moments=np.array(laugh_moments)*0.05
#print(laugh_moments)

def myfunc(lst):
    ret = []
    a = b = lst[0]                           # a and b are range's bounds

    for el in lst[1:]:
        if el == b+1: b = el                 # range grows
        else:                                # range ended
            ret.append(a if a==b else (a,b)) # is a single or a range?
            a = b = el                       # let's start again with a single
    ret.append(a if a==b else (a,b))         # corner case for last single/range
    return ret

ret=myfunc(laugh_moments)

for i in range (0,len(ret)):
    if(isinstance( ret[i], int )):
        current=ret[i]
        start_time=current*0.05
        end_time=start_time+0.3
        print(round(start_time,2),round(end_time,2))
    else:
        temp=list(ret[i])
        start_time=round(temp[0]*0.05,2)
        end_time=round(temp[1]*0.05+0.3,2)
        print(start_time,end_time)

0.0 0.45
0.4 1.1
2.4 3.5
3.65 4.8
5.7 6.25
6.05 6.35
7.75 8.3
9.45 9.95
9.95 10.65
11.95 12.45
12.35 12.95
12.8 13.65
13.9 14.7
14.9 15.65
16.25 17.25
23.05 23.35
23.25 23.55
23.35 23.9
33.8 34.2
45.3 45.75
46.1 46.75
47.5 47.9
50.1 51.4
51.85 52.3
52.55 53.5
53.75 54.4
54.9 55.6
58.9 59.3
59.85 60.15
60.3 60.6
60.5 61.5
62.4 63.4
63.25 63.65
63.45 65.0
64.95 65.7
66.4 67.0
66.8 67.35
69.1 69.9
70.2 70.55
70.45 71.1
71.5 72.25
72.05 73.05
73.4 73.8
74.0 75.3
78.85 80.35
80.3 80.85
80.65 81.3
81.85 82.4
82.5 83.05
82.85 83.45
83.3 83.6
84.3 84.85
84.7 85.25
86.15 87.05
87.45 87.95
87.75 88.35
88.2 88.6
88.9 90.05
91.3 92.2
92.15 93.8
93.6 95.5
95.5 96.6
96.45 97.6
97.55 98.2
98.5 98.8
99.25 99.85
99.9 100.4
101.45 102.55
103.8 105.2
105.05 105.35
105.25 106.0
106.1 106.9
106.85 107.7
107.75 108.85
109.0 109.6
109.55 110.05
110.1 110.7
110.65 113.1
113.0 113.85
113.95 114.75
114.95 115.3
115.2 115.85
116.6 117.1
117.0 118.4
118.3 118.75
119.2 120.0
119.95 121.2
121.1 122.2
122.2 124.85
1

In [18]:
#-----------Audio5----------------
with open("S_Deception_audio5.csv", 'rt') as f:
    data = []
    try:
        reader = csv.reader(f)
        for row in reader:
            data.append(np.array(row))                   
    finally:
        f.close()
    
data = np.array(data,float) 
prediction=clf.predict(data)

laugh_moments=[]
for i in range (0,len(data)):
    if(prediction[i]==1):
        laugh_moments.append(i)
        
#laugh_moments=np.array(laugh_moments)*0.05
#print(laugh_moments)

def myfunc(lst):
    ret = []
    a = b = lst[0]                           # a and b are range's bounds

    for el in lst[1:]:
        if el == b+1: b = el                 # range grows
        else:                                # range ended
            ret.append(a if a==b else (a,b)) # is a single or a range?
            a = b = el                       # let's start again with a single
    ret.append(a if a==b else (a,b))         # corner case for last single/range
    return ret

ret=myfunc(laugh_moments)

for i in range (0,len(ret)):
    if(isinstance( ret[i], int )):
        current=ret[i]
        start_time=current*0.05
        end_time=start_time+0.3
        print(round(start_time,2),round(end_time,2))
    else:
        temp=list(ret[i])
        start_time=round(temp[0]*0.05,2)
        end_time=round(temp[1]*0.05+0.3,2)
        print(start_time,end_time)

0.1 1.1
0.9 2.3
2.2 3.45
3.5 4.3
4.5 5.55
5.95 6.65
6.55 7.3
7.1 7.75
7.7 8.15
8.8 9.4
11.55 12.85
15.3 16.0
17.65 18.15
18.65 20.05
19.85 20.6
30.0 30.6
33.4 34.05
35.4 35.9
40.2 41.4
41.75 43.0
43.05 43.5
43.55 44.25
44.3 44.7
44.6 45.1
46.05 47.65
47.65 48.0
48.2 48.75
49.3 49.65
49.85 50.45
50.35 51.3
51.25 51.6
51.45 52.35
52.15 52.55
52.5 53.8
53.6 54.3
54.3 54.7
54.65 55.95
56.25 57.3
57.8 58.6
58.95 62.6
63.0 63.3
64.75 65.15
64.95 65.45
66.3 67.15
68.6 69.5
71.7 72.25
72.2 73.05
73.2 74.85
75.85 76.85
77.4 77.7
78.0 78.7
78.55 79.05
79.0 79.55
79.55 79.85
80.2 80.5
80.95 81.5
81.6 82.8
82.9 83.25
83.1 83.55
83.4 83.9
83.75 84.25
85.3 85.8
85.85 86.4
87.9 88.9
88.75 89.3
90.8 92.0
92.4 92.85
94.0 95.4
95.25 95.55
95.7 96.0
95.8 96.1
96.15 96.45
96.25 96.65
96.8 97.7
97.8 98.1
98.3 99.1
98.95 99.75
100.0 100.4
100.4 101.75
101.55 101.85
101.65 102.15
102.05 103.0
103.35 104.5
104.35 105.0
104.9 106.2
106.95 107.35
107.45 108.2
108.55 109.2
109.65 110.5
111.0 111.3
111.2 111.85
1

In [19]:
#-----------Audio6----------------
with open("S_Deception_audio6.csv", 'rt') as f:
    data = []
    try:
        reader = csv.reader(f)
        for row in reader:
            data.append(np.array(row))                   
    finally:
        f.close()
    
data = np.array(data,float) 
prediction=clf.predict(data)

laugh_moments=[]
for i in range (0,len(data)):
    if(prediction[i]==1):
        laugh_moments.append(i)
        
#laugh_moments=np.array(laugh_moments)*0.05
#print(laugh_moments)

def myfunc(lst):
    ret = []
    a = b = lst[0]                           # a and b are range's bounds

    for el in lst[1:]:
        if el == b+1: b = el                 # range grows
        else:                                # range ended
            ret.append(a if a==b else (a,b)) # is a single or a range?
            a = b = el                       # let's start again with a single
    ret.append(a if a==b else (a,b))         # corner case for last single/range
    return ret

ret=myfunc(laugh_moments)

for i in range (0,len(ret)):
    if(isinstance( ret[i], int )):
        current=ret[i]
        start_time=current*0.05
        end_time=start_time+0.3
        print(round(start_time,2),round(end_time,2))
    else:
        temp=list(ret[i])
        start_time=round(temp[0]*0.05,2)
        end_time=round(temp[1]*0.05+0.3,2)
        print(start_time,end_time)

0.1 0.4
0.2 0.9
0.75 1.7
1.85 2.65
2.45 2.75
2.8 3.55
3.35 4.25
4.3 5.0
5.8 7.35
7.8 8.7
8.85 9.6
9.7 10.55
10.4 10.7
10.9 11.9
12.1 12.5
12.45 13.7
13.5 13.8
14.2 15.3
18.2 18.75
18.7 19.2
20.05 20.55
22.15 22.7
23.5 24.2
26.6 27.0
27.0 27.8
27.6 27.95
27.75 28.55
28.4 28.7
28.55 28.9
28.7 29.05
29.0 29.5
29.75 30.95
30.9 32.2
32.3 32.8
32.9 33.45
33.3 33.75
33.55 34.1
35.05 36.55
36.4 36.7
36.85 37.25
37.4 37.75
37.6 37.9
38.1 39.05
39.75 40.1
40.0 40.55
42.95 43.45
43.25 43.6
43.65 44.1
44.85 45.45
46.5 46.8
46.6 46.9
46.7 47.05
48.05 48.35
48.2 48.55
49.7 50.1
51.1 51.45
51.8 52.35
52.7 53.0
53.25 54.15
54.55 55.0
54.8 55.1
56.5 56.9
56.7 57.25
57.6 58.55
58.4 58.7
58.6 59.3
59.25 60.55
61.2 61.75
61.55 61.85
61.75 62.3
62.1 62.4
62.2 62.55
63.55 63.85
63.8 64.3
64.2 64.5
64.35 64.75
64.55 64.85
65.15 66.1
65.9 66.25
66.05 66.45
66.3 66.95
66.75 67.1
67.3 68.85
68.7 69.05
69.15 69.55
69.6 69.9
69.95 70.45
70.6 71.0
70.95 72.15
71.95 72.25
72.6 73.35
73.25 75.15
75.6 75.9
75.85 76.3

In [20]:
#-----------Audio7----------------
with open("S_Deception_audio7.csv", 'rt') as f:
    data = []
    try:
        reader = csv.reader(f)
        for row in reader:
            data.append(np.array(row))                   
    finally:
        f.close()
    
data = np.array(data,float) 
prediction=clf.predict(data)

laugh_moments=[]
for i in range (0,len(data)):
    if(prediction[i]==1):
        laugh_moments.append(i)
        
#laugh_moments=np.array(laugh_moments)*0.05
#print(laugh_moments)

def myfunc(lst):
    ret = []
    a = b = lst[0]                           # a and b are range's bounds

    for el in lst[1:]:
        if el == b+1: b = el                 # range grows
        else:                                # range ended
            ret.append(a if a==b else (a,b)) # is a single or a range?
            a = b = el                       # let's start again with a single
    ret.append(a if a==b else (a,b))         # corner case for last single/range
    return ret

ret=myfunc(laugh_moments)

for i in range (0,len(ret)):
    if(isinstance( ret[i], int )):
        current=ret[i]
        start_time=current*0.05
        end_time=start_time+0.3
        print(round(start_time,2),round(end_time,2))
    else:
        temp=list(ret[i])
        start_time=round(temp[0]*0.05,2)
        end_time=round(temp[1]*0.05+0.3,2)
        print(start_time,end_time)

0.05 0.4
0.3 0.65
0.45 0.8
1.25 2.65
2.55 3.35
3.9 6.3
6.55 7.1
8.6 9.15
9.8 10.15
10.5 10.85
10.7 11.3
11.75 12.05
11.85 12.15
12.3 13.5
14.1 14.75
14.85 15.95
18.05 18.55
22.85 23.65
23.6 24.3
24.1 24.5
24.35 25.2
25.05 26.5
26.35 26.8
27.4 27.95
27.9 28.7
28.8 29.25
29.05 29.55
29.9 30.3
30.15 30.5
30.35 30.65
32.4 32.95
34.9 36.1
37.25 37.55
37.45 37.8
40.15 40.7
41.45 41.75
41.6 42.05
41.85 42.2
43.65 44.1
43.9 44.25
44.05 44.35
45.25 45.55
46.0 46.55
47.0 47.45
47.25 47.55
47.5 48.05
48.25 49.05
48.95 49.25
49.4 49.7
49.9 50.2
50.1 50.45
50.25 50.85
51.4 51.7
51.5 51.8
51.7 52.7
52.5 53.25
53.2 53.65
53.45 53.75
53.7 54.0
54.2 54.5
54.3 54.65
55.25 55.55
55.35 55.65
55.45 55.95
56.25 56.75
56.55 57.1
60.05 60.35
60.6 61.1
61.25 62.25
62.15 63.55
64.45 65.0
66.2 66.5
68.2 68.55
69.15 69.6
69.95 70.5
70.7 71.05
71.15 71.45
71.55 71.85
71.65 72.4
72.6 73.4
73.25 73.55
73.35 74.35
74.65 74.95
74.8 76.65
76.6 77.0
76.8 77.15
77.35 78.0
78.4 79.05
79.1 79.4
79.95 80.6
80.65 81.2
81.0 8

In [21]:
#-----------Audio8----------------
with open("S_Deception_audio8.csv", 'rt') as f:
    data = []
    try:
        reader = csv.reader(f)
        for row in reader:
            data.append(np.array(row))                   
    finally:
        f.close()
    
data = np.array(data,float) 
prediction=clf.predict(data)

laugh_moments=[]
for i in range (0,len(data)):
    if(prediction[i]==1):
        laugh_moments.append(i)
        
#laugh_moments=np.array(laugh_moments)*0.05
#print(laugh_moments)

def myfunc(lst):
    ret = []
    a = b = lst[0]                           # a and b are range's bounds

    for el in lst[1:]:
        if el == b+1: b = el                 # range grows
        else:                                # range ended
            ret.append(a if a==b else (a,b)) # is a single or a range?
            a = b = el                       # let's start again with a single
    ret.append(a if a==b else (a,b))         # corner case for last single/range
    return ret

ret=myfunc(laugh_moments)

for i in range (0,len(ret)):
    if(isinstance( ret[i], int )):
        current=ret[i]
        start_time=current*0.05
        end_time=start_time+0.3
        print(round(start_time,2),round(end_time,2))
    else:
        temp=list(ret[i])
        start_time=round(temp[0]*0.05,2)
        end_time=round(temp[1]*0.05+0.3,2)
        print(start_time,end_time)

1.1 1.85
2.15 2.85
3.65 4.4
4.45 5.05
4.85 5.5
5.3 5.6
5.4 6.8
7.6 8.15
8.05 8.65
8.85 9.4
10.2 10.6
10.4 11.05
11.0 11.3
11.95 12.45
12.6 12.9
12.95 13.5
13.35 13.9
14.8 15.2
15.75 17.05
17.25 18.4
19.25 19.55
19.8 20.3
20.3 20.8
20.9 21.2
21.0 21.8
22.0 23.25
23.55 24.85
24.9 25.2
25.5 26.4
26.25 26.65
26.7 27.0
26.85 27.15
27.6 29.1
28.95 29.25
30.2 32.05
32.65 33.05
34.15 34.45
35.4 35.7
42.15 42.5
45.0 45.65
47.55 48.15
49.2 49.7
49.95 50.25
50.45 50.9
55.15 55.7
55.55 55.9
57.3 57.9
58.0 60.1
60.15 63.4
63.7 66.5
66.75 72.05
72.7 73.6
73.75 74.1
74.1 74.55
75.2 77.75
77.55 77.85
77.8 79.8
79.6 79.95
79.75 80.4
80.2 81.0
81.05 83.05
82.85 85.2
85.4 85.7
85.5 85.85
86.6 87.2
87.35 89.75
89.6 90.0
89.8 91.2
91.0 92.25
92.3 92.95
92.75 94.0
95.25 95.7
95.8 96.3
96.3 97.1
96.9 97.2
97.15 97.6
97.6 97.9
97.7 98.35
98.5 99.05
98.95 99.25
99.25 100.05
99.85 100.35
100.4 102.1
102.75 103.05
103.85 105.25
106.0 106.3
106.45 106.8
107.9 108.35
108.45 108.8
108.6 108.9
110.0 111.4
111.75 112

In [22]:
#-----------Audio9----------------
with open("S_Deception_audio9.csv", 'rt') as f:
    data = []
    try:
        reader = csv.reader(f)
        for row in reader:
            data.append(np.array(row))                   
    finally:
        f.close()
    
data = np.array(data,float) 
prediction=clf.predict(data)

laugh_moments=[]
for i in range (0,len(data)):
    if(prediction[i]==1):
        laugh_moments.append(i)
        
#laugh_moments=np.array(laugh_moments)*0.05
#print(laugh_moments)

def myfunc(lst):
    ret = []
    a = b = lst[0]                           # a and b are range's bounds

    for el in lst[1:]:
        if el == b+1: b = el                 # range grows
        else:                                # range ended
            ret.append(a if a==b else (a,b)) # is a single or a range?
            a = b = el                       # let's start again with a single
    ret.append(a if a==b else (a,b))         # corner case for last single/range
    return ret

ret=myfunc(laugh_moments)

for i in range (0,len(ret)):
    if(isinstance( ret[i], int )):
        current=ret[i]
        start_time=current*0.05
        end_time=start_time+0.3
        print(round(start_time,2),round(end_time,2))
    else:
        temp=list(ret[i])
        start_time=round(temp[0]*0.05,2)
        end_time=round(temp[1]*0.05+0.3,2)
        print(start_time,end_time)

0.0 0.3
0.2 1.0
0.9 1.45
1.25 2.05
2.25 2.65
2.45 2.75
2.8 3.8
3.75 4.1
4.0 4.3
4.2 5.8
5.7 6.0
5.8 6.45
6.35 7.15
7.0 8.0
7.85 10.45
10.25 11.0
10.95 12.4
12.5 13.6
13.85 14.25
14.15 14.65
14.55 14.85
14.65 15.0
14.8 15.1
14.9 15.8
15.95 16.4
16.65 16.95
16.8 18.7
18.5 18.8
18.6 19.45
19.25 19.6
19.85 20.15
20.2 20.65
20.9 21.55
21.35 21.65
21.5 21.8
21.6 21.9
21.7 22.0
22.05 22.35
22.15 22.45
22.35 22.95
22.95 23.3
23.3 24.0
23.8 24.1
23.9 24.8
25.15 25.6
26.2 27.05
26.95 27.25
27.1 27.8
27.6 28.0
28.5 29.05
28.85 29.15
29.15 29.75
30.35 30.65
30.6 31.35
31.2 31.9
31.8 32.2
32.0 32.35
32.5 34.0
33.8 34.4
34.25 34.9
35.05 35.95
36.25 36.95
36.9 37.55
37.6 38.55
38.45 40.4
40.35 41.35
41.15 41.45
41.5 42.5
42.3 43.6
43.45 43.75
43.65 44.2
44.2 45.15
45.3 45.6
45.4 45.7
46.05 46.35
46.4 47.25
47.1 47.4
47.2 48.15
48.0 49.15
49.0 49.3
49.1 49.9
49.75 50.05
49.85 57.25
57.2 58.35
58.35 58.75
58.55 58.9
58.7 59.5
59.3 59.95
60.0 62.05
63.3 63.85
63.85 64.2
64.85 65.75
65.75 66.15
66.25 68.

In [23]:
#-----------Audio10----------------
with open("S_Deception_audio10.csv", 'rt') as f:
    data = []
    try:
        reader = csv.reader(f)
        for row in reader:
            data.append(np.array(row))                   
    finally:
        f.close()
    
data = np.array(data,float) 
prediction=clf.predict(data)

laugh_moments=[]
for i in range (0,len(data)):
    if(prediction[i]==1):
        laugh_moments.append(i)
        
#laugh_moments=np.array(laugh_moments)*0.05
#print(laugh_moments)

def myfunc(lst):
    ret = []
    a = b = lst[0]                           # a and b are range's bounds

    for el in lst[1:]:
        if el == b+1: b = el                 # range grows
        else:                                # range ended
            ret.append(a if a==b else (a,b)) # is a single or a range?
            a = b = el                       # let's start again with a single
    ret.append(a if a==b else (a,b))         # corner case for last single/range
    return ret

ret=myfunc(laugh_moments)

for i in range (0,len(ret)):
    if(isinstance( ret[i], int )):
        current=ret[i]
        start_time=current*0.05
        end_time=start_time+0.3
        print(round(start_time,2),round(end_time,2))
    else:
        temp=list(ret[i])
        start_time=round(temp[0]*0.05,2)
        end_time=round(temp[1]*0.05+0.3,2)
        print(start_time,end_time)

0.1 1.1
0.9 1.2
1.0 1.35
1.55 2.45
2.75 3.3
3.1 3.45
4.0 4.3
4.1 4.45
4.8 5.35
5.15 5.5
5.8 6.2
6.05 7.1
8.1 8.65
8.8 9.8
9.65 10.25
10.2 11.15
12.35 14.1
14.2 14.5
14.45 15.1
14.9 15.5
15.3 15.75
15.55 15.85
16.0 16.4
16.45 17.05
16.85 17.15
17.1 17.4
17.2 18.15
17.95 18.25
18.65 19.15
19.8 20.35
20.5 21.6
22.15 22.45
22.25 22.6
22.5 22.8
22.6 23.5
23.45 24.25
24.05 24.35
24.9 25.75
26.1 26.65
27.7 28.25
28.1 28.75
29.75 30.05
29.85 30.15
30.25 30.7
30.7 31.15
31.05 31.35
32.05 32.5
32.5 33.15
32.95 33.6
33.4 33.75
34.15 35.0
35.05 35.55
35.65 36.65
38.25 38.8
39.05 39.6
39.6 40.45
40.4 46.85
47.5 49.7
49.9 50.2
50.0 50.45
51.25 52.65
52.65 53.15
53.05 54.2
54.35 55.95
56.1 56.9
56.95 57.85
57.7 58.55
58.35 58.95
58.9 60.05
60.2 62.3
62.65 63.55
63.65 63.95
63.85 64.15
64.0 64.3
64.95 67.8
67.9 68.95
69.1 76.1
75.9 77.0
77.0 80.1
80.0 82.85
83.1 83.45
83.25 83.65
83.85 84.95
84.85 86.05
86.05 87.45
87.35 99.45
99.5 105.0
104.8 113.65
113.5 115.4
115.2 116.8
116.8 117.1
116.9 117.95
11